In [1]:
import requests
import pandas as pd
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing as mp
import os
from datetime import datetime
import numpy as np

# """____for input and output path____"""

global root_path
global input_path
global output_path
global output_file_name
global url_count
global url_processed
global url_failed
global f_data


# root_path = os.path.dirname(os.path.abspath(__file__))
root_path = os.getcwd()
input_path = os.path.join(root_path, "input")
output_path = os.path.join(root_path, "output")
input_file = 'urls.csv'
output_file_name = 'output_'

f_data = pd.DataFrame(columns=['URL','Status','Number of redirects','Redirect Chain','Final URL'])
try:
    list_urls = pd.read_csv(os.path.join(input_path, input_file),
                                        delimiter="\t")['url'].values
except:
    list_urls = pd.read_csv(os.path.join(input_path, output_file),
                                    delimiter="\t")['URL'].values
url_count = len(list_urls)
print(url_count)
url_processed = 0
url_failed = 0
#____initializing date & time for output file name
today = datetime.now().strftime('%m%d_%H%M')

def get_status_code(url):
    global root_path
    global input_path
    global output_path
    global output_file_name
    global url_count
    global url_processed
    global url_failed
    global f_data
    url_count-=1
    url_processed +=1
        
    try:
        r = requests.get(url)
        print("Processing " + str(url))

        if len(r.history) > 0:
            chain = ""
            code = r.history[0].status_code
            final_url = r.url
            for resp in r.history:
                chain += resp.url + " | "
            print(str(url_processed)+" URLs Processed | "+str(url_count)+" Remaining | "+str(url_failed)+" Failed")
            df = pd.DataFrame([[url,str(code),str(len(r.history)),chain,final_url]], columns=['URL','Status','Number of redirects','Redirect Chain','Final URL'])
            f_data = f_data.append(df)
            f_data.to_csv(os.path.join(output_path, output_file_name+today+".csv"), sep='\t', encoding='utf-8', index=False)
            return [url,str(code),str(len(r.history)),chain,final_url]
        
        else:
            df = pd.DataFrame([[url,str(r.status_code),'n/a','n/a','n/a']], columns=['URL','Status','Number of redirects','Redirect Chain','Final URL'])
            f_data = f_data.append(df)
            f_data.to_csv(os.path.join(output_path, output_file_name+today+".csv"), sep='\t', encoding='utf-8', index=False)
            return [url,str(r.status_code),'n/a','n/a','n/a']
        
    except requests.ConnectionError:
        print("Error: failed to connect.")
        url_failed +=1
        print(str(url_processed)+" URLs Processed | "+str(url_count)+" Remaining | "+str(url_failed)+" Failed")
        df = pd.DataFrame([[url,'0','n/a','n/a','n/a']], columns=['URL','Status','Number of redirects','Redirect Chain','Final URL'])
        f_data = f_data.append(df)
        f_data.to_csv(os.path.join(output_path, output_file_name+today+".csv"), sep='\t', encoding='utf-8', index=False)
        return [url,'0','n/a','n/a','n/a']


def final_redirect(urls):
    data = []
    for i in urls:
        efr = get_status_code(i)
        data.append(efr)
                
    return data
 
    
#____Preparing arguments for multiprocessing
cpu_count = 4
print(cpu_count)
pool = ThreadPool(cpu_count)
list_split = np.array_split(list_urls, cpu_count)
general_data = pool.map(final_redirect, list_split)


#____merging list 
data_combined = []
for i in general_data:
    for j in i:
        data_combined.append(j)
        
#____transforming list data to dataframe

dataf = pd.DataFrame(data_combined,columns=['URL','Status','Number of redirects','Redirect Chain','Final URL'])
                         
#___writing csv file "tab separated"
dataf.to_csv(os.path.join(output_path, "final_"+output_file_name+today+".csv"), sep='\t', encoding='utf-8', index=False)




15679
4
Processing https://rainbowresource.com/
Processing http://www.rainbowresource.com/
5 URLs Processed | 15674 Remaining | 0 Failed
Processing http://www.rainbowresource.com/category/4101/Exploring-the-Building-Blocks-of-Science-Book-K.html
6 URLs Processed | 15673 Remaining | 0 Failed
Processing http://www.rainbowresource.com/category/11572/Memoria-Press-Science.html
7 URLs Processed | 15672 Remaining | 0 Failed
Processing http://www.rainbowresource.com/category/5539/Layers-of-Learning-Year-3-Units.html
8 URLs Processed | 15671 Remaining | 0 Failed
Error: failed to connect.
9 URLs Processed | 15670 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/1480/Classical-Poetry---Traditional-Poetry.html
10 URLs Processed | 15669 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/5677/Theory-for-Young-Musicians.html
11 URLs Processed | 15668 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/6066/Treasure-Island-Books.html
12 

Processing http://www.rainbowresource.com/category/12358/Double-Digit-Decoders.html
66 URLs Processed | 15613 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/7908/Friendly-Anatomy-Bovine-Skeletal-System.html
67 URLs Processed | 15612 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/14424/Ice-Cream-Accessories.html
68 URLs Processed | 15611 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/12115/GRADE-1-BIBLE.html
69 URLs Processed | 15610 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/134/Weekly-Lesson-Plan-Books.html
70 URLs Processed | 15609 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/5782/Music-for--CD-series.html
71 URLs Processed | 15608 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/1892/Core-Skills-Spelling.html
72 URLs Processed | 15607 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/2143/180-Days-of-Language.

Processing http://www.rainbowresource.com/category/3528/Crinkle-Cuddlers.html
127 URLs Processed | 15552 Remaining | 1 Failed
Processing http://www.rainbowresource.com/product/032671/Make-A-Dinosaur---T-Rex.html
128 URLs Processed | 15551 Remaining | 1 Failed
Processing http://www.rainbowresource.com/product/009760/Cats-Scramble-Squares.html
129 URLs Processed | 15550 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/11892/WordBuild.html
130 URLs Processed | 15549 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/9102/Colors-and-Shapes.html
Processing http://www.rainbowresource.com/category/6606/Writers-Toolbox-Series.html
132 URLs Processed | 15547 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/371/Comprehensive-Curriculum-of-Basic-Skills.html
133 URLs Processed | 15546 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/6547/ARTS--and--CRAFTS-SKILLS.html
134 URLs Processed | 15545 Remaining | 1 F

Processing http://www.rainbowresource.com/category/12505/Take-10-Games.html
188 URLs Processed | 15491 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/11453/Zaner-Bloser-Broken-Midline-Paper-Reams.html
189 URLs Processed | 15490 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/11648/Pocket-Genius---History.html
190 URLs Processed | 15489 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/9723/Farmer-Boy-Memoria-Press.html
191 URLs Processed | 15488 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/3693/Goal-Setting-Banners.html
192 URLs Processed | 15487 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/8948/Naya-Nuki.html
193 URLs Processed | 15486 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/14166/Science-Graphic-Organizer.html
194 URLs Processed | 15485 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/5911/Much-Ado-About-No

Processing http://www.rainbowresource.com/category/3751/Devotions-For-Boys.html
248 URLs Processed | 15431 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/10912/Kid-agami.html
249 URLs Processed | 15430 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/4465/Young-Nation-Solves-Its-Problems.html
250 URLs Processed | 15429 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/14217/Collage---Decoupage.html
251 URLs Processed | 15428 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/5747/VIOLIN---FIDDLE---OTHER-STRINGED-INSTRUMENTS.html
252 URLs Processed | 15427 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/9907/Djeco-Puzzles.html
253 URLs Processed | 15426 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/2153/New-Elementary-Math-2-Gr-8.html
254 URLs Processed | 15425 Remaining | 1 Failed
Processing http://www.rainbowresource.com/category/13909/SwitchBoar

...

Processing https://rainbowresource.com/pdfs/products/prod007575_smpl0.pdf
Processing http://www.rainbowresource.com/category/10532/Caroline-Books.html
15398 URLs Processed | 281 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/1279/Language-Arts-Games--and--Books.html
15399 URLs Processed | 280 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/8350/Skip-Counting---Start-to-Finish.html
15400 URLs Processed | 279 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/5463/Refill-Origami-Paper.html
15401 URLs Processed | 278 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/1084/New-Coat-for-Anna.html
15402 URLs Processed | 277 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/10515/Air-Dough.html
15403 URLs Processed | 276 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/8152/Saxon-Math-Intermediate-Grade-4.html
15404 URLs Processed | 275 Remaining | 5 Failed
P

Processing http://www.rainbowresource.com/category/15103/Colonial-Times-Books--and--Resources.html
15459 URLs Processed | 220 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/4659/GeoSafari-Talking-Globe.html
15460 URLs Processed | 219 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/13181/Family-Memory-Games.html
15461 URLs Processed | 218 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/10489/Fridge-Art.html
15462 URLs Processed | 217 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/9778/Pacon-Elementary-Composition-Books-DNealian---Zaner-Bloser-Rulings.html
15463 URLs Processed | 216 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/14187/Outdoor-Activity-Pads.html
15464 URLs Processed | 215 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/14032/Days-of-Decision.html
15465 URLs Processed | 214 Remaining | 5 Failed
Processing http://www.rainbowresou

Processing http://www.rainbowresource.com/category/9746/Adventures-of-Tom-Sawyer-Saxon-Homeschool-Literature-Study-Guides.html
15520 URLs Processed | 159 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/11633/Screwtape-Letters-w--Screwtape-Proposes-Toast.html
15521 URLs Processed | 158 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/10612/Father-Goose-Gr.-9-12.html
15522 URLs Processed | 157 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/9330/Route-66-Gr.-6-8.html
15523 URLs Processed | 156 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/1707/Bat-House-Kit.html
15524 URLs Processed | 155 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/11465/Primary-Math-Common-Core-Edition-Grade-2.html
15525 URLs Processed | 154 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/556/Activity-Idea-Books.html
15526 URLs Processed | 153 Remaining | 5 Failed
Processin

Processing http://www.rainbowresource.com/category/4067/Algebra-II-CLE-Math.html
15580 URLs Processed | 99 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/4284/Readers-in-Residence-Vol.-1-Book-Study-Resources.html
15581 URLs Processed | 98 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/13105/Brown-Dog-Electronics-Kits.html
15582 URLs Processed | 97 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/10929/Magnetic-Tiles.html
15583 URLs Processed | 96 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/7724/Required-Art-Supplies-for-4-5-Book-2-listed-individually.html
15584 URLs Processed | 95 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/14050/True-Books---World-War-II.html
15585 URLs Processed | 94 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/2533/Flexitables.html
15586 URLs Processed | 93 Remaining | 5 Failed
Processing http://www.rainbowresour

Processing http://www.rainbowresource.com/category/14233/Crayola-and-174-Silly-Scents-Mini-Twistable-Crayons.html
15641 URLs Processed | 38 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/11100/Chinese-Flash-Cards-Wink-to-Learn.html
15642 URLs Processed | 37 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/446/America-Integrated-Curriculum-Part-II.html
15643 URLs Processed | 36 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/11259/Incentive-Charts,-Sticker,-and-Sets---Dinosaur.html
15644 URLs Processed | 35 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/6152/Wall-Calendar-Accessories.html
15645 URLs Processed | 34 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/2581/Wild-Berries--and--Fruits-Field-Guides.html
15646 URLs Processed | 33 Remaining | 5 Failed
Processing http://www.rainbowresource.com/category/8262/Traditional-Cursive.html
15647 URLs Processed | 32 Remaini